In [1]:
import sys
sys.path.insert(0, "/notebooks/pipenv")
sys.path.insert(0, "/notebooks/nebula3_database")
sys.path.insert(0, "/notebooks/")
from PIL import Image
import requests
import visual_genome.local as vg
import json
import copy
import subprocess

import numpy as np
import torch
import spacy
import nltk
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
from sentence_transformers import SentenceTransformer
from database.arangodb import DatabaseConnector
from config import NEBULA_CONF


In [2]:
nltk.download('wordnet')
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe("spacy_wordnet", after='tagger', config={'lang': nlp.lang})

VG_DATA = '/storage/vg_data'
IPC_COLLECTION = 'ipc_relations_spice'
RECALL_COLLECTION = 'ipc_recall_spice'
GLOBAL_TOKENS_COLLECTION = 's3_global_tokens'

class PIPELINE:
    def __init__(self):
        config = NEBULA_CONF()
        self.db_host = config.get_database_host()
        self.database = config.get_playground_name()
        self.gdb = DatabaseConnector()
        self.db = self.gdb.connect_db(self.database)

pipeline = PIPELINE()
ipc_data = json.load(open('/storage/ipc_data/paragraphs_v1.json','r'))
def get_sc_graph(id):
    return vg.get_scene_graph(id, images=VG_DATA,
                    image_data_dir=VG_DATA+'/by-id/',
                    synset_file=VG_DATA+'/synsets.json')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
class GTBaseGenerator:
    def __init__(self):
        self.pipeline = PIPELINE()

    def get_image_id_from_collection(self, id,collection=GLOBAL_TOKENS_COLLECTION):
        results = {}
        query = 'FOR doc IN {} FILTER doc.image_id == {} RETURN doc'.format(collection,id)
        #print(query)
        cursor = self.pipeline.db.aql.execute(query)
        for doc in cursor:
            results.update(doc)
        return results
    
    def get_structure(self, id):
        sg = get_sc_graph(id)
        global_doc = self.get_image_id_from_collection(id)
        if not global_doc:
            print("Couldn't find global tokens for id {}".format(id))
            return
        rc_doc = {
            'image_id': id,
            'url': sg.image.url            
        }
        for (k,v) in global_doc.items():
            if k.startswith('global'):
                rc_doc[k]=copy.copy(v)
        rois = []
        for obj in sg.objects:
            obj_dic = {
                'GT': list(zip(obj.names,[1.0]*len(obj.names)))
            }
            attr_dic = {
                'GT': list(zip(obj.attributes,[1.0]*len(obj.attributes)))
            }
            obj_doc = {                
                'objects': obj.names,
                'attributes': obj.attributes,
                'bbox': [obj.x, obj.y, obj.x+obj.width, obj.y+obj.height]              
                }
            rois.append(obj_doc)
        rc_doc['rois']=rois

        return rc_doc


In [27]:
base_gen = GTBaseGenerator()

In [28]:
# 2367862 in ipc
id = 2367862
rc = base_gen.get_structure(id)

In [30]:
rc['global_captions']

{'blip': 'a room with a computer and a chair and a chair'}

In [18]:
sg.objects[0].__dict__

{'id': 3871935,
 'x': 0,
 'y': 3,
 'width': 498,
 'height': 368,
 'names': ['office'],
 'synsets': [office.n.01 - place of business where professional or clerical duties are performed],
 'attributes': []}

In [ ]:
rc = vg.get_all_region_descriptions(data_dir=VG_DATA)


In [ ]:
list(zip([],[1.0]*0))